<a href="https://colab.research.google.com/github/dantrainor9/Animal_Image_Classification/blob/main/Animal_Image_Classification_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install timm

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn, optim
import timm
import torchvision
from torchvision import datasets, transforms
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split

In [14]:
path = '/content/drive/MyDrive/Personal Projects/Animal Image Classification/dataset'

In [15]:
def get_data_loaders(filename, batch_size, train = False):
  if train:
    my_transforms = transforms.Compose([
          transforms.RandomHorizontalFlip(),
          transforms.RandomVerticalFlip(),
          transforms.ColorJitter(brightness=0.3),
          transforms.RandomRotation(30),
          transforms.Resize(256),
          transforms.CenterCrop(224),
          transforms.ToTensor(),
          transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
    ])
    all_data = datasets.ImageFolder(filename,transform=my_transforms)
    train_len = int(len(all_data) * 0.7)
    val_len = int((len(all_data) - train_len)/2)
    test_len = int(len(all_data) - train_len-val_len)
    train_data, val_data, test_data = random_split(all_data, [train_len, val_len, test_len])
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
    return(train_loader, train_len, all_data)
#generating DataLoader for train data

  else:
    my_transforms = transforms.Compose([
          transforms.Resize(256),
          transforms.CenterCrop(224),
          transforms.ToTensor(),
          transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
    ])
    all_data = datasets.ImageFolder(filename, transform=my_transforms)
    train_len = int(len(all_data)*0.7)
    val_len = int((len(all_data)-train_len)/2)
    test_len = int(len(all_data)-train_len-val_len)
    train_data, val_data, test_data = random_split(all_data, [train_len,val_len,test_len])
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=2)
    return(val_loader, val_len, test_loader, test_len)
#generating DataLoader for test and validation data

In [16]:
def get_classes(filename):
  all_data = datasets.ImageFolder(filename)
  return all_data.classes

In [17]:
#instnatiating data loaders
(train_loader, train_len, all_data) = get_data_loaders(path, 64, train=True)
(val_loader, val_len, test_loader, test_len) = get_data_loaders(path, 64, train=False)
classes = get_classes(path)

In [18]:
dataloaders = {'train':train_loader,
               'val':val_loader}
dataset_sizes = {'train_len':train_len,
                 'val_len':val_len}

In [19]:
print(len(train_loader))
print(len(val_loader))
print(len(test_loader))
print(train_len,test_len,val_len)
#visualizing split sizes

69
15
15
4389 941 940


In [20]:
class AnimalClassifier(torch.nn.Module):

  def __init__(self):
    super(AnimalClassifier, self).__init__()
    self.conv1 = torch.nn.Conv2d(3, 10, 5)
    self.conv2 = torch.nn.Conv2d(10, 45, 3)
    self.fc1 = torch.nn.Linear(45*10*10, 500)
    self.fc2 = torch.nn.Linear(120,84)
    self.fc3 = torch.nn.Linear(84,151)

  def forward(self,x):
    x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
    x = F.max_pood2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

  def num_flat_features(x):
    size = x.size()[1:]
    num_features=1
    for s in size:
        num_features *= s
    return num_features

model = AnimalClassifier()

In [21]:
loss_fn = torch.nn.CrossEntropyLoss()

dummy_out = torch.rand(10, 151)
dummy_label = torch.tensor([1,6,8,3,9,4,2,5,7,4])